In [1]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

model_name = "stanford-crfm/BioMedLM"
model = GPT2ForSequenceClassification.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at stanford-crfm/BioMedLM and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
import torch

In [35]:
def get_decision(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class_idx = outputs.logits.argmax(dim=-1).item()
#     print(predicted_class_idx)
# #     decision_map = {0: "yes", 1: "no", 2: "maybe"}
# #     decision = decision_map.get(predicted_class_idx, "unknown")
    
    return predicted_class_idx

In [4]:
from datasets import load_dataset
dataset = load_dataset("pubmed_qa", "pqa_labeled")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Dataset pubmed_qa downloaded and prepared to /root/.cache/huggingface/datasets/pubmed_qa/pqa_labeled/1.0.0/2e65addecca4197502cd10ab8ef1919a47c28672f62d7abac7cc9afdcf24fb2d. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
MAX_TOKENS = 512

def truncate_text_to_fit(text, max_tokens=MAX_TOKENS):
    token_count = len(tokenizer.tokenize(text))
    if token_count > max_tokens - 2:  
        truncated_text = " ".join(text.split()[:-1])
        while len(tokenizer.tokenize(truncated_text)) > max_tokens - 2:
            truncated_text = " ".join(truncated_text.split()[:-1])
    else:
        truncated_text = text
        
    return truncated_text

In [6]:
def create_input_text(sample):
    input_text = "question: "+sample['question'] + " context: " + "".join(sample['context']['contexts'])
    return {'input_text': input_text}

modified_dataset = dataset['train'].map(create_input_text)

  0%|          | 0/1000 [00:00<?, ?ex/s]

In [37]:
import pandas as pd

# Assuming you have the functions and the modified_dataset loaded

def save_to_csv(indexes, pubids, results, filename='results.csv'):
    """Utility function to save the data to CSV."""
    df = pd.DataFrame({
        'Index': indexes,
        'PubID': pubids,
        'Result': results
    })
    # If file doesn't exist, write header, otherwise append without header
    if not pd.io.common.file_exists(filename):
        df.to_csv(filename, index=False)
    else:
        df.to_csv(filename, mode='a', header=False, index=False)

results = []
pubids = []
indexes = []

for idx, text in enumerate(modified_dataset):
    print(idx + 1)
    
    indexes.append(idx + 1)
    pubids.append(text['pubid'])
    results.append(get_decision(truncate_text_to_fit(text['input_text'])))
    
    if (idx + 1) % 100 == 0:  # Check if it's the 100th, 200th, etc. iteration
        save_to_csv(indexes, pubids, results)
        # Clear the lists to start collecting the next batch of results
        results.clear()
        pubids.clear()
        indexes.clear()

# Save any remaining results (if less than 100) at the end
if results:
    save_to_csv(indexes, pubids, results)

1
0


NameError: name 'decision' is not defined

In [9]:
data = pd.read_csv('/kaggle/working/results.csv')

In [11]:
data = data.drop('Index', axis=1)

In [15]:
pubids= data["PubID"].values.tolist()

In [17]:
actual={}

for text in dataset['train']:
    if(text['pubid'] in pubids):
        actual[text['pubid']]= text['final_decision']

In [20]:
predictions= dict(data.values)

In [21]:
from sklearn.metrics import accuracy_score, classification_report


if set(predictions.keys()) != set(actual.keys()):
    raise ValueError("Keys in predictions and ground truth do not match!")

In [28]:
y_pred = [predictions[key] for key in sorted(predictions.keys())]
y_true = [actual[key] for key in sorted(actual.keys())]

# Compute accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc}")

# Compute detailed classification report
report = classification_report(y_true, y_pred)
print(report)

Accuracy: 0.495
              precision    recall  f1-score   support

       maybe       0.00      0.00      0.00       110
          no       0.30      0.17      0.22       338
         yes       0.54      0.79      0.64       552

    accuracy                           0.49      1000
   macro avg       0.28      0.32      0.29      1000
weighted avg       0.40      0.49      0.43      1000



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
